***
    
Demo Tensorflow for PINN (Physics-Informed Neural Network)
    
***

    Version: 2023-02-24
    
    Author : Nicholas Sung Wei Yong, Wong Jian Cheng, Ooi Chin Chun, Abhishek Gupta

    Contain:
    
    1. PINN for solving Linearized Burgers equation 
    
    2. Optimize PINN using CMA-ES on the Tensorflow framework
       -  as described in Nicholas Sung Wei Yong, Jian Cheng Wong, Pao-Hsiung Chiu, Abhishek Gupta, Chinchun Ooi, Yew-Soon Ong
       "Neuroevolution Surpasses Stochastic Gradient Descent for Physics-Informed Neural Networks" arXiv preprint arXiv:2212.07624 (2022).

Import Libraries

In [1]:
%matplotlib inline
import matplotlib.pylab as plt 
from matplotlib import cm

import os
import math
import numpy as np
import pandas as pd
import cma
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint, Callback
import tensorflow.keras.backend as K

from IPython.display import clear_output
from time import time, gmtime, strftime, localtime

import warnings
warnings.filterwarnings("ignore")

2022-12-13 19:51:07.965853: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from scipy import dot, exp, log, sqrt, floor, ones, randn, zeros_like, Inf, argmax, argmin, eye, outer, zeros
from scipy import array, power, diag, cos, sin, pi
from scipy.linalg import expm, cholesky, solve, norm, eig
from scipy.stats import multivariate_normal

import seaborn as sns

## run on CPU...
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# disable eager excution
tf.compat.v1.disable_eager_execution()



### 1. PINN for kdv equation

#### 1.0. Data

#### 1.1. Geometry, PDE parameter & BC

In [3]:
vis = 0.02
c = 1

# initial condition
k = 2
m = 10

# spatial domain
x_l, x_u = -1.5, 6.5

# time domain: 0 - t_T
t_T = 2.0

#### 1.2. Sampling plan

In [4]:
sim = pd.read_csv('linear.csv')
sim = sim[sim.x <= 4.5]
batch_X = np.vstack([sim.t.values, sim.x.values]).T
y_train = sim[['u']].values

#### 1.3. PDE-NN

In [5]:
# specify NN model + Physis
def create_nn(nodes):

    # input layers -> split into (t, x)
    inputs = layers.Input(shape=(2,))
    t, x = layers.Lambda( lambda k: tf.split(k, num_or_size_splits=2, axis=1))(inputs)

    # hidden layers
    hidden_1 = layers.Dense(nodes, activation='tanh')(layers.Concatenate()([t, x]))
    hidden_2 = layers.Dense(nodes, activation='tanh')(hidden_1)
    hidden_l = layers.Dense(nodes, activation='tanh')(hidden_2)

    # output layers
    u = layers.Dense(1, name="u", use_bias=False)(hidden_l)

    # axillary PDE outputs
    u_t = K.gradients(u, t)[0]
    u_x = K.gradients(u, x)[0]
    u_xx = K.gradients(u_x, x)[0]
    pde_residuals = u_t + c*u_x - vis*u_xx

    # PDE residuals (u_t + c*u_x - vis*u_xx = 0)
    pde_mse = tf.compat.v1.losses.mean_squared_error(labels=tf.zeros_like(pde_residuals), predictions=pde_residuals)

    # initial conditions: u(0, x) = m*exp(-(k*x)**2)
    x_0, u_0 = tf.boolean_mask(x, tf.equal(t, 0)), tf.boolean_mask(u, tf.equal(t, 0))
    ic = m*tf.math.exp(tf.math.negative(tf.math.square(k*x_0)))
    ic_mse = tf.compat.v1.losses.mean_squared_error(labels=ic, predictions=u_0)


    # initiate model
    nn = models.Model(inputs=inputs, outputs=u)

    # optimizer
    optimizer = tf.keras.optimizers.Adam(0.05)

    # compile model with [physics] loss
    nn.compile(loss = compute_physic_loss(pde_mse, ic_mse),
               optimizer = optimizer,
               metrics = [compute_pde_loss(pde_mse), compute_ic_loss(ic_mse)])

    # pathway to variables inside NN
    insiders = [u, pde_mse, ic_mse]
    eval_ins = K.function([nn.input, K.learning_phase()], insiders)   # evaluation function

    return (nn, eval_ins)

In [6]:
# define loss function (PDE + IC loss)
def compute_physic_loss(pde_mse, ic_mse):
    def physic_loss(y_true, y_pred):
        pde_loss  = pde_mse # PDE
        ic_loss   = ic_mse  # IC
        return pde_loss + ic_loss
    return physic_loss

# define loss function (PDE loss)
def compute_pde_loss(pde_mse):
    def pde_loss(y_true, y_pred): return pde_mse
    return pde_loss # return a function

# define loss function (IC loss)
def compute_ic_loss(ic_mse):
    def ic_loss(y_true, y_pred): return ic_mse
    return ic_loss # return a function

# define loss function (data loss)
def compute_data_loss(z):
    def data_loss(y_true, y_pred):
        p_data = tf.equal(z, 1)
        return tf.compat.v1.losses.mean_squared_error(labels=tf.boolean_mask(y_true, p_data), predictions=tf.boolean_mask(y_pred, p_data))
    return data_loss

### 3. Optimize PINN with [Neuroevolution / xNES]

In [7]:
# initiate NN model (& pathway to internal values)
n_nodes = 10
nn, eval_ins = create_nn(n_nodes)

# initial weights
w0 = np.array([])
nn_weights = nn.get_weights()
for _g in nn_weights: w0 = np.append(w0, _g.flatten())

nn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 [(None, 1), (None, 1 0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 2)            0           lambda[0][0]                     
                                                                 lambda[0][1]                     
__________________________________________________________________________________________________
dense (Dense)                   (None, 10)           30          concatenate[0][0]            

2022-12-13 19:51:11.457294: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-12-13 19:51:11.473556: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-13 19:51:11.473579: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ihpcgs-03
2022-12-13 19:51:11.473584: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ihpcgs-03
2022-12-13 19:51:11.473692: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.141.3
2022-12-13 19:51:11.473711: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.141.3
2022-12-13 19:51:11.473715: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.141.3
2022-12-13 19:51:11.474031: I tensorflow/core/platform/cpu_fe

In [8]:
# nn weights structure 
nn_weights = nn.get_weights()
nn_wlen = len(nn_weights)
nn_wshape = [_w.shape for _w in nn_weights]
nn_widx = np.cumsum([len(_w.flatten()) for _w in nn_weights])
nn_nweights = nn_widx[-1]

In [9]:

def nn_fitness(_weights):
    _weights = np.split(_weights, nn_widx[:-1])
    _weights = [_weights[i].reshape(nn_wshape[i]) for i in range(nn_wlen)]
    nn.set_weights(_weights)
    _x = batch_X
    _, pde_mse, ic_mse = eval_ins(_x)
    fitness = (pde_mse + ic_mse)
    return fitness



In [10]:
x, es = cma.fmin2(nn_fitness, w0, 0.01, options={'ftarget':1e-04 ,'popsize':50, 'maxiter':360, 'verb_disp':1})

(25_w,50)-aCMA-ES (mu_w=14.0,w_1=14%) in dimension 260 (seed=1078325, Tue Dec 13 19:51:11 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     50 1.406853580474854e+01 1.0e+00 9.66e-03  1e-02  1e-02 0:00.6
    2    100 1.381969642639160e+01 1.0e+00 9.41e-03  9e-03  9e-03 0:01.1
    3    150 1.358181190490723e+01 1.0e+00 9.23e-03  9e-03  9e-03 0:01.6
    4    200 1.345410728454590e+01 1.0e+00 9.10e-03  9e-03  9e-03 0:02.1
    5    250 1.322073650360107e+01 1.0e+00 9.00e-03  9e-03  9e-03 0:02.6
    6    300 1.312499046325684e+01 1.0e+00 8.96e-03  9e-03  9e-03 0:03.1
    7    350 1.286690902709961e+01 1.0e+00 8.95e-03  9e-03  9e-03 0:03.6
    8    400 1.266655826568604e+01 1.0e+00 8.98e-03  9e-03  9e-03 0:04.1
    9    450 1.261105537414551e+01 1.0e+00 9.04e-03  9e-03  9e-03 0:04.6
   10    500 1.228941345214844e+01 1.0e+00 9.13e-03  9e-03  9e-03 0:05.1
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
   11    550 1.225606155395508

  102   5100 8.800496459007263e-01 1.1e+00 1.61e-01  2e-01  2e-01 0:52.8
  103   5150 1.093739509582520e+00 1.1e+00 1.63e-01  2e-01  2e-01 0:53.3
  104   5200 7.162806987762451e-01 1.1e+00 1.65e-01  2e-01  2e-01 0:53.9
  105   5250 7.831873893737793e-01 1.1e+00 1.67e-01  2e-01  2e-01 0:54.4
  106   5300 9.513932466506958e-01 1.1e+00 1.69e-01  2e-01  2e-01 0:54.8
  107   5350 8.191636800765991e-01 1.1e+00 1.70e-01  2e-01  2e-01 0:55.3
  108   5400 8.320074081420898e-01 1.1e+00 1.71e-01  2e-01  2e-01 0:55.8
  109   5450 8.333946466445923e-01 1.1e+00 1.73e-01  2e-01  2e-01 0:56.3
  110   5500 5.842761993408203e-01 1.1e+00 1.74e-01  2e-01  2e-01 0:56.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
  111   5550 7.774230837821960e-01 1.1e+00 1.76e-01  2e-01  2e-01 0:57.3
  112   5600 6.650999188423157e-01 1.1e+00 1.77e-01  2e-01  2e-01 0:57.9
  113   5650 5.797455906867981e-01 1.1e+00 1.79e-01  2e-01  2e-01 0:58.3
  114   5700 5.100704431533813e-01 1.1e+00 1.80e-01 

  205  10250 4.663576781749725e-01 1.2e+00 1.54e-01  2e-01  2e-01 1:45.8
  206  10300 5.970602631568909e-01 1.2e+00 1.54e-01  2e-01  2e-01 1:46.3
  207  10350 6.045348644256592e-01 1.2e+00 1.53e-01  2e-01  2e-01 1:46.9
  208  10400 3.694593608379364e-01 1.2e+00 1.53e-01  2e-01  2e-01 1:47.4
  209  10450 4.221165180206299e-01 1.2e+00 1.52e-01  2e-01  2e-01 1:47.9
  210  10500 3.961148858070374e-01 1.2e+00 1.51e-01  1e-01  2e-01 1:48.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
  211  10550 4.920518994331360e-01 1.2e+00 1.51e-01  1e-01  2e-01 1:49.0
  212  10600 3.427827656269073e-01 1.2e+00 1.51e-01  1e-01  2e-01 1:49.6
  213  10650 4.577803015708923e-01 1.2e+00 1.51e-01  1e-01  2e-01 1:50.1
  214  10700 2.635003030300140e-01 1.2e+00 1.51e-01  1e-01  2e-01 1:50.6
  215  10750 2.744615077972412e-01 1.2e+00 1.51e-01  1e-01  2e-01 1:51.2
  216  10800 2.937824726104736e-01 1.2e+00 1.50e-01  1e-01  2e-01 1:51.7
  217  10850 4.395264089107513e-01 1.2e+00 1.50e-01 

  308  15400 2.697140276432037e-01 1.3e+00 1.17e-01  1e-01  1e-01 2:39.0
  309  15450 1.860858201980591e-01 1.3e+00 1.17e-01  1e-01  1e-01 2:39.5
  310  15500 2.546798288822174e-01 1.3e+00 1.16e-01  1e-01  1e-01 2:40.0
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
  311  15550 2.309516370296478e-01 1.3e+00 1.16e-01  1e-01  1e-01 2:40.5
  312  15600 2.424561381340027e-01 1.3e+00 1.15e-01  1e-01  1e-01 2:41.0
  313  15650 2.116218507289886e-01 1.3e+00 1.15e-01  1e-01  1e-01 2:41.5
  314  15700 1.433404386043549e-01 1.3e+00 1.14e-01  1e-01  1e-01 2:42.0
  315  15750 1.697653979063034e-01 1.3e+00 1.14e-01  1e-01  1e-01 2:42.5
  316  15800 1.704879105091095e-01 1.3e+00 1.14e-01  1e-01  1e-01 2:43.0
  317  15850 1.343992650508881e-01 1.3e+00 1.13e-01  1e-01  1e-01 2:43.5
  318  15900 1.939006745815277e-01 1.3e+00 1.12e-01  1e-01  1e-01 2:44.0
  319  15950 2.299163341522217e-01 1.3e+00 1.12e-01  1e-01  1e-01 2:44.6
  320  16000 1.889723837375641e-01 1.3e+00 1.11e-01 